# English to POrtuguese translator

In [1]:
import collections
import logging
import os
import pathlib
import re
import string
import sys
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf

In [2]:
logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings


In [8]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']


Dataset ted_hrlr_translate downloaded and prepared to /home/rachel/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0. Subsequent calls will reuse this data.


In [9]:
for be_examples, en_examples in train_examples.batch(3).take(1):
    for be in be_examples.numpy():
        print(be.decode('utf-8'))

    print()

    for en in en_examples.numpy():
        print(en.decode('utf-8'))


e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .
mas e se estes fatores fossem ativos ?
mas eles não tinham a curiosidade de me testar .

and when you improve searchability , you actually take away the one advantage of print , which is serendipity .
but what if it were active ?
but they did n't test for curiosity .


## Text tokenization & detokenization

In [7]:
## downloading text.BertTokenizer, a tokenizer made just for data like this
model_name = "ted_hrlr_translate_pt_en_converter"
tf.keras.utils.get_file(
    f"{model_name}.zip",
    f"https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip",
    cache_dir='.', cache_subdir='', extract=True
)


188416/184801 [==============================] - 0s 2us/step


'./ted_hrlr_translate_pt_en_converter.zip'

### loading a saved tokenizing model  model from tensorflows library

In [10]:
#loading the tokenizer model we just downloaded
tokenizers = tf.saved_model.load(model_name)


The tokenize method converts a batch of strings to a padded-batch of token IDs. This method splits punctuation, lowercases and unicode-normalizes the input before tokenizing. That standardization is not visible here because the input data is already standardized.

**example of tokenzing with this model**

In [11]:
for en in en_examples.numpy():
    print(en.decode('utf-8'))

encoded = tokenizers.en.tokenize(en_examples)

for row in encoded.to_list():
    print(row)


and when you improve searchability , you actually take away the one advantage of print , which is serendipity .
but what if it were active ?
but they did n't test for curiosity .
[2, 72, 117, 79, 1259, 1491, 2362, 13, 79, 150, 184, 311, 71, 103, 2308, 74, 2679, 13, 148, 80, 55, 4840, 1434, 2423, 540, 15, 3]
[2, 87, 90, 107, 76, 129, 1852, 30, 3]
[2, 87, 83, 149, 50, 9, 56, 664, 85, 2512, 15, 3]


**now lets take tockens and trslate back to words**

In [12]:
round_trip = tokenizers.en.detokenize(encoded)
for line in round_trip.numpy():
    print(line.decode('utf-8'))


and when you improve searchability , you actually take away the one advantage of print , which is serendipity .
but what if it were active ?
but they did n ' t test for curiosity .


### after tokenizing you need to vectorize (embed)

In [13]:
# this function converts  the sentaces to token ids then into encoded vectors [2,5] --> [0,1,0,0,1]

def tokenize_pairs(pt, en):
    pt = tokenizers.pt.tokenize(pt)
    # Convert from ragged to dense, padding with zeros.
    pt = pt.to_tensor()

    en = tokenizers.en.tokenize(en)
    # Convert from ragged to dense, padding with zeros.
    en = en.to_tensor()
    return pt, en

### here is a funtion that, processes, shuffles and batches the data (Pipeline!):

In [14]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

ef make_batches(ds):
    return (
      ds
        #grabs the data set and saves it in memory
      .cache()
        #randomly shuffles data i think the buffer size is how many are swapped becasue "For perfect shuffling, a buffer size greater than or equal to the full size of the dataset is required."
      .shuffle(BUFFER_SIZE)
        #Combines consecutive elements of this dataset into batches.
      .batch(BATCH_SIZE)
        # performs a function across the whole data set (in this case it tokenizez with the function we wrote)
           #Performance can often be improved by setting num_parallel_calls so that map will use multiple threads to process elements. If deterministic order isn't required.
           #it is  the number elements to process asynchronously in parallel.  If the value tf.data.AUTOTUNE is used, then the number of parallel calls is set dynamically based on available CPU.
      .map(tokenize_pairs, num_parallel_calls=tf.data.AUTOTUNE)
        #Most dataset input pipelines should end with a call to prefetch. This allows later elements to be prepared while the current element is being processed. This often improves latency and throughput, at the cost of using additional memory to store prefetched elements.
      .prefetch(tf.data.AUTOTUNE))


train_batches = make_batches(train_examples)
val_batches = make_batches(val_examples)


SyntaxError: invalid syntax (<ipython-input-14-70e9a56e80bd>, line 4)

## possitional codeing